In [59]:
import sys
import os

import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
print(tf.__version__)
from datetime import datetime

from tensorflow.keras.models import load_model
from keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

2.15.0


In [47]:
# Path to your HDF5 file
model_path = 'models/speed/20240317'

# Load the model
speed_model = load_model(model_path)

# Now the model is ready to use
speed_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0     

In [48]:
# # Path to your HDF5 file
# weights_path = 'models/angle/angle_navigation.h5'

# def nvidia_model():
#     model = Sequential(name='Nvidia_A_Model')
#     model.add(tf.keras.applications.mobilenet_v2.MobileNetV2(include_top = False, weights="imagenet", input_shape=(200, 200, 3)))
#     model.add(tf.keras.layers.GlobalAveragePooling2D())
#     model.add(Dense(1, activation = 'sigmoid'))
#     model.layers[0].trainable = False

#     model.compile(optimizer=tf.optimizers.RMSprop(lr=0.01), loss='binary_crossentropy', metrics='accuracy')

#     return model


# # Create the model
# angle_model = nvidia_model()

# # Load the weights
# angle_model.load_weights(weights_path)

# Path to your HDF5 file
angle_path = 'models/angle/20240317'

# Load the model
angle_model = load_model(angle_path)

# Now the model is ready to use
angle_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0     

In [2]:

# # Construct the absolute path to the module's directory
# # os.path.abspath returns the absolute path of the given path
# # os.path.join combines paths in a way that's safe for any operating system
# module_dir = os.path.abspath(os.path.join('models', 'maggie'))
# print(module_dir)

# # Add the module's directory to sys.path
# if module_dir not in sys.path:
#     sys.path.append(module_dir)

# Now you can import the Model class
from model import Model

# Use the Model class as needed
model_instance = Model()


In [7]:
# # FUNCTION TO READ TESTING DATA
# def read_dataset_test(img_path):
#     img = tf.io.read_file(img_path)
#     img = tf.image.decode_png(img,channels=3)
#     img = tf.cast(img, tf.float32) 
#     # print(img.shape)
#     # img = tf.image.resize(img,[224,224])
#     # img = tf.reshape(img,[1,224,224,3])
#     return img

# def my_imread(image_path):
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     return image

# def preprocess(image):
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV) / 255.0
#     height, _, _ = image.shape
#     image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
#     image = cv2.GaussianBlur(image, (3,3), 0)
#     image = cv2.resize(image, (200,200)) # input image size (200,66) Nvidia model
#     image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
#     return image

In [54]:
def my_imread(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def img_preprocess(image):
    # height, _, _ = image.shape
    # image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (224,224)) # input image size (200,66) Nvidia model
    # image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    return image

In [55]:
def predict(image):
    angles = np.arange(17)*5+50
    image = img_preprocess(image)
    image_reshaped = image.reshape((1, 224, 224, 3))
    # print(image_reshaped.shape)
    pred_speed = speed_model.predict(image_reshaped)[0]
    # print(pred_speed)
    # speed = pred_speed[0].astype(int)*35
    angle = angle_model.predict(image_reshaped)[0]
    pred_angle = angles[np.argmax(angle)]
    # angle = angles[np.argmax(pred_angle)]
    # print('angle:', angle,'speed:', speed)
    
    return pred_angle, pred_speed

In [58]:
# TO CREATE PREDICTIONS OF THE TEST DATA
predictions = []
test_dir = os.path.abspath(os.path.join('test_data','test_data'))

for i in range(1,1021):
    img_path = f"{test_dir}/{i}.png"
    img = my_imread(img_path)
    # print(img.shape)
    # angle_pred, speed_pred = model_instance.predict(img)
    
    angle_pred, speed_pred = predict(img)
    print(angle_pred, speed_pred)

    if abs(1-speed_pred[0]) < abs(speed_pred[0]-0):
            speed_pred[0] = 1
    else:
        speed_pred[0] = 0



    print([(angle_pred-50)/80, speed_pred[0]])
    predictions.append([(angle_pred-50)/80, speed_pred[0]])
    


1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 79ms/step
95 [4.1320772e-07]
[0.5625, 0.0]
1/1 [==============================] - 0s 38ms/step
110 [0.99992293]
[0.75, 1.0]
1/1 [==============================] - 0s 39ms/step
65 [0.9451244]
[0.1875, 1.0]
1/1 [==============================] - 0s 43ms/step
65 [0.9999709]
[0.1875, 1.0]
1/1 [==============================] - 0s 39ms/step
60 [0.9968474]
[0.125, 1.0]
1/1 [==============================] - 0s 35ms/step
105 [0.999976]
[0.6875, 1.0]
1/1 [==============================] - 0s 34ms/step
115 [0.9999922]
[0.8125, 1.0]
1/1 [==============================] - 0s 36ms/step
110 [0.9999966]
[0.75, 1.0]
1/1 [==============================] - 0s 35ms/step
95 [2.9190945e-05]
[0.5625, 0.0]
1/1 [==============================] - 0s 37ms/step
120 [0.04989028]
[0.875, 0.0]
1/1 [==============================] - 0s 33ms/step
90 [3.954487e-06]
[0.5, 0.0]
1/1 [==============================] - 0s 33ms/step
110 [0.00098399]
[0.75, 0.0]
1/1 [===============

In [60]:
print(predictions[:5])

[[0.5625, 0.0], [0.75, 1.0], [0.1875, 1.0], [0.1875, 1.0], [0.125, 1.0]]


In [61]:
timestamp = datetime.now().strftime('%Y%m%d')

df_pred = pd.DataFrame(predictions, columns=['angle', 'speed'])

# Add an 'image_id' column as the first column, set to be the index + 1
df_pred['image_id'] = np.arange(1, len(predictions) + 1)

# Reorder the DataFrame to have 'image_id' as the first column
df_pred = df_pred[['image_id', 'angle', 'speed']]

filepath = f'submission_{timestamp}.csv'
# Save the DataFrame to a CSV file, without including the index
df_pred.to_csv(filepath, index=False)

## Combined

In [5]:
# Path to your HDF5 file
combined_model_path = 'models/combined/02-1.47.hdf5'

# Load the model
combined_model = load_model(combined_model_path)

# Now the model is ready to use
combined_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['rescaling[0][0]']           
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                          

In [13]:
def my_imread(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def img_preprocess(image):
    # height, _, _ = image.shape
    # image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (224,224)) # input image size (200,66) Nvidia model
    # image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    # image = (image - 127.5) / 127.5
    return image

In [39]:
def predict(image):
    angles = np.arange(17)*5+50
    image = img_preprocess(image)
    image_reshaped = image.reshape((1, 224, 224, 3))

    pred_angle, pred_speed  = combined_model.predict(image_reshaped)

    angle = angles[np.argmax(pred_angle)]
    return angle, pred_speed[0]

In [43]:
# TO CREATE PREDICTIONS OF THE TEST DATA
predictions = []
test_dir = os.path.abspath(os.path.join('test_data','test_data'))

for i in range(1,1021):
    img_path = f"{test_dir}/{i}.png"
    img = my_imread(img_path)
    # print(img.shape)
    # angle_pred, speed_pred = model_instance.predict(img)
    
    angle_pred, speed_pred = predict(img)
    # print(angle_pred, speed_pred)

    if abs(1-speed_pred[0]) < abs(speed_pred[0]-0):
            speed_pred[0] = 1
    else:
        speed_pred[0] = 0


    predictions.append([(angle_pred - 50)/80, speed_pred[0]])
    # # angle_pred = angle
    print([(angle_pred-50)/80, speed_pred[0]])

1/1 [==============================] - 0s 69ms/step


[0.5625, 0.0]
1/1 [==============================] - 0s 58ms/step
[0.75, 1.0]
1/1 [==============================] - 0s 77ms/step
[0.1875, 1.0]
1/1 [==============================] - 0s 139ms/step
[0.1875, 1.0]
1/1 [==============================] - 0s 111ms/step
[0.125, 1.0]
1/1 [==============================] - 0s 128ms/step
[0.625, 1.0]
1/1 [==============================] - 0s 103ms/step
[0.6875, 1.0]
1/1 [==============================] - 0s 129ms/step
[0.75, 1.0]
1/1 [==============================] - 0s 87ms/step
[0.5625, 0.0]
1/1 [==============================] - 0s 126ms/step
[0.6875, 1.0]
1/1 [==============================] - 0s 70ms/step
[0.5, 0.0]
1/1 [==============================] - 0s 154ms/step
[0.75, 0.0]
1/1 [==============================] - 0s 78ms/step
[0.5, 1.0]
1/1 [==============================] - 0s 88ms/step
[0.5625, 0.0]
1/1 [==============================] - 0s 55ms/step
[0.125, 1.0]
1/1 [==============================] - 0s 82ms/step
[0.8125, 1.0]
1/1 

In [44]:
df_pred = pd.DataFrame(predictions, columns=['angle', 'speed'])

# Add an 'image_id' column as the first column, set to be the index + 1
df_pred['image_id'] = np.arange(1, len(predictions) + 1)

# Reorder the DataFrame to have 'image_id' as the first column
df_pred = df_pred[['image_id', 'angle', 'speed']]

# Save the DataFrame to a CSV file, without including the index
df_pred.to_csv('submission_mobilenet_combined.csv', index=False)

In [45]:

combined_model_path = 'models/speed/20240317'

# Load the model
combined_model = load_model(combined_model_path)

# Now the model is ready to use
combined_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0     

In [109]:
angle_path = 'models/angle/mobilenet_v1_20240321'
angle_model = load_model(angle_path)
angle_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 160, 160, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 160, 160, 3)       0         
 da)                                                             
                                                                 
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                             

In [117]:
def preprocess(image_path):
    image = tf.keras.utils.load_img(image_path)
    im = tf.cast(image, tf.float32)
    im = tf.image.resize(im, [160, 160])
    im = tf.expand_dims(im, axis=0)
    return im

In [118]:
def predict(image_path):
    angles = np.arange(17)*5+50
    image = preprocess(image_path)

    angle = angle_model.predict(image)[0]
    pred_angle = angles[np.argmax(angle)]
    # angle = angles[np.argmax(pred_angle)]
    # print('angle:', angle,'speed:', speed)
    
    return pred_angle

In [132]:
# TO CREATE PREDICTIONS OF THE TEST DATA
predictions = []
test_dir = os.path.abspath(os.path.join('test_data','test_data'))

for i in range(1,1021):
    img_path = f"{test_dir}/{i}.png"
    
    angle_pred = predict(img_path)
    # print(angle_pred, speed_pred)
    print(i, angle_pred, (angle_pred-50)/80)
    # predictions.append([(angle_pred - 50)/80, speed_pred[0]])
    # # # angle_pred = angle
    # print([(angle_pred-50)/80, speed_pred[0]])

1/1 [==============================] - 0s 48ms/step
1 130 1.0
1/1 [==============================] - 0s 25ms/step
2 60 0.125
1/1 [==============================] - 0s 28ms/step


3 110 0.75
1/1 [==============================] - 0s 26ms/step
4 95 0.5625
1/1 [==============================] - 0s 21ms/step
5 100 0.625
1/1 [==============================] - 0s 20ms/step
6 50 0.0
1/1 [==============================] - 0s 24ms/step
7 65 0.1875
1/1 [==============================] - 0s 23ms/step
8 60 0.125
1/1 [==============================] - 0s 33ms/step
9 130 1.0
1/1 [==============================] - 0s 22ms/step
10 65 0.1875
1/1 [==============================] - 0s 24ms/step
11 125 0.9375
1/1 [==============================] - 0s 20ms/step
12 60 0.125
1/1 [==============================] - 0s 19ms/step
13 125 0.9375
1/1 [==============================] - 0s 20ms/step
14 50 0.0
1/1 [==============================] - 0s 21ms/step
15 110 0.75
1/1 [==============================] - 0s 21ms/step
16 60 0.125
1/1 [==============================] - 0s 21ms/step
17 130 1.0
1/1 [==============================] - 0s 23ms/step
18 55 0.0625
1/1 [===========================

In [1]:
from datetime import datetime
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [2]:
def preprocess(image_path):
    image = tf.keras.utils.load_img(image_path, target_size=(160,192))
    im = tf.cast(image, tf.float32)
    im = tf.image.resize(im, [160, 160])
    im = tf.expand_dims(im, axis=0)
    return im

In [33]:
def predict_angle(image_path, model):
    angles = [100.0,105.0,110.0,115.0,120.0,125.0,130.0,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0]
    image = preprocess(image_path)

    angle = model.predict(image)[0]
    predicted_value = angle[0] * 80 + 50

    # Find the closest value in angles to predicted_value
    pred_angle = min(angles, key=lambda x: abs(x - predicted_value))
    pred = (pred_angle-50)/80
    print(pred)
    return angle[0]

In [3]:
timestamp = datetime.now().strftime('%Y%m%d')

In [10]:
angle_path = f'models/angle/resnet_reg_best_{timestamp}'
angle_model = load_model(angle_path)

2024-03-26 13:28:32.863876: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-26 13:28:32.864188: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-26 13:28:32.864208: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-26 13:28:32.864460: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-26 13:28:32.864845: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
test_dir = os.path.abspath('test_data/test_data')
file_list = os.listdir(test_dir)
predictions = []

In [28]:
timestamp

'20240326'

In [30]:
predictions=[]

In [34]:
for file_name in sorted(file_list, key=lambda x: int(x.split('.')[0])):
    img_path = os.path.join(test_dir, file_name)
    image_id = int(file_name.split('.')[0])
    
    angle_pred = predict_angle(img_path, angle_model)
    # print(angle_pred)
    print(f"Image ID: {image_id}, Angle: {angle_pred}")
    predictions.append([image_id, angle_pred])

df_pred = pd.DataFrame(predictions, columns=['image_id', 'angle'])
df_pred.to_csv(f'submission_{timestamp}_1.csv', index=False)

1/1 [==============================] - 0s 376ms/step
0.6875
Image ID: 1, Angle: 0.6845247745513916
1/1 [==============================] - 0s 56ms/step
0.9375
Image ID: 2, Angle: 0.9564573764801025
1/1 [==============================] - 0s 68ms/step
0.0625
Image ID: 3, Angle: 0.09352642297744751
1/1 [==============================] - 0s 40ms/step
0.1875
Image ID: 4, Angle: 0.2136867642402649
1/1 [==============================] - 0s 44ms/step
0.0625
Image ID: 5, Angle: 0.07409206032752991
1/1 [==============================] - 0s 41ms/step
0.8125
Image ID: 6, Angle: 0.8034019470214844
1/1 [==============================] - 0s 41ms/step
0.75
Image ID: 7, Angle: 0.7752295732498169
1/1 [==============================] - 0s 41ms/step
1.0
Image ID: 8, Angle: 1.0765318870544434
1/1 [==============================] - 0s 66ms/step
0.8125
Image ID: 9, Angle: 0.8080395460128784
1/1 [==============================] - 0s 40ms/step
0.625
Image ID: 10, Angle: 0.6193625926971436
1/1 [=================

In [32]:
predictions

[[1, 0.6875],
 [2, 0.9375],
 [3, 0.0625],
 [4, 0.1875],
 [5, 0.0625],
 [6, 0.8125],
 [7, 0.75],
 [8, 1.0],
 [9, 0.8125],
 [10, 0.625],
 [11, 0.4375],
 [12, 0.8125],
 [13, 0.375],
 [14, 0.625],
 [15, 0.1875],
 [16, 0.75],
 [17, 0.625],
 [18, 0.875],
 [19, 0.4375],
 [20, 0.1875],
 [21, 0.9375],
 [22, 0.1875],
 [23, 0.1875],
 [24, 0.875],
 [25, 0.75],
 [26, 0.9375],
 [27, 0.75],
 [28, 0.6875],
 [29, 0.75],
 [30, 0.875],
 [31, 0.375],
 [32, 1.0],
 [33, 0.6875],
 [34, 0.875],
 [35, 0.6875],
 [36, 0.8125],
 [37, 0.5],
 [38, 1.0],
 [39, 0.6875],
 [40, 0.9375],
 [41, 0.8125],
 [42, 0.6875],
 [43, 0.125],
 [44, 0.5],
 [45, 0.0625],
 [46, 0.8125],
 [47, 0.9375],
 [48, 0.6875],
 [49, 0.75],
 [50, 1.0],
 [51, 0.6875],
 [52, 0.625],
 [53, 0.875],
 [54, 1.0],
 [55, 0.8125],
 [56, 0.6875],
 [57, 0.625],
 [58, 0.75],
 [59, 0.0625],
 [60, 0.8125],
 [61, 0.875],
 [62, 0.375],
 [63, 1.0],
 [64, 0.9375],
 [65, 1.0],
 [66, 0.8125],
 [67, 0.625],
 [68, 0.625],
 [69, 0.25],
 [70, 0.75],
 [71, 0.3125],
 [72, 